# Data Aquisition

We will use __PushShift__ to extract the data of subrredit _India_ from 7/01/2020 till 7/04/2020.

In [11]:
import pandas as pd
import requests

import json
import csv
import time
import datetime

def getPushshiftData(after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission?&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) 
    numComms = subm['num_comments']
    permalink = subm['permalink']
    try:
        selftext=subm['selftext']
    except KeyError:
        selftext="NaN"
    over_18=subm['over_18']
    
    
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair,over_18,selftext))
    subStats[sub_id] = subData
    
#Subreddit to query.
sub="india"
#before and after dates
before = "1586284088" #April 7st
after = "1577895896"  #January 7st 

subCount = 0
subStats = {}

# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
        
    # Calls getPushshiftData() with the created date of the last submission
    
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(after, before, sub)
    
# Storing data in csv file

def updateSubs_file():
    upload_count = 0
    
    file = "post_data.csv" #filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair","Over 18","Selftext"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()


In [13]:
data=pd.read_csv('post_data.csv')
data.shape

(44027, 12)

Extracting Comments using praw and adding it to the already scraped Pushshift dataset above.

In [2]:
import praw
import pandas as pd


# Enter your credentials here
reddit=praw.Reddit(client_id='xxxx',
client_secret='xxxxx',
user_agent='xxxx',
password='xxxx')

# Read the data extracted from Pushshitft
data=pd.read_csv('post_data.csv')

# Add an empty column "Comments"
data['Comments']=""

n=len(data)
coms=[]

def getComments(id):
    
    #url of the reddit post
    url="https://www.reddit.com/r/india/comments/"+id
    
    submission=reddit.submission(url=url)
    
    #escape "Load more comments"
    submission.comments.replace_more(limit=0)
    comments=submission.comments.list()
    
    count=1
 
    coms.clear()
    
    for com in comments:
        
        if count<11:   #getting top 10 comments
            coms.append(com.body)
            count+=1
        else:
            break  
          
    return list(coms) #coms contains 10 comments of a post
        
for i in range (0,n):
    #get the post_id from the dataset
    id=data['Post ID'][i]
    data.at[i,'Comments']=getComments(id)
    

In [ ]:
#saving the dataset for further operations.
data=pd.to_csv('data_final.csv')